<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語学習済みモデルによる自然言語処理

### 準備（関連ライブラリのインストール）

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece

## 1. RoBERTa による文章中の空欄埋め



### モデルのダウンロード

In [57]:
from transformers import T5Tokenizer, RobertaForMaskedLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")

### 問題文の作成

In [ ]:
# original text
#text = "4年に1度オリンピックは開かれる。"
text = ""

# prepend [CLS]
text = "[CLS]" + text

# tokenize
tokens = tokenizer.tokenize(text)
print(tokens)

# mask a token
masked_idx = 5
tokens[masked_idx] = tokenizer.mask_token
print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']

### 穴埋め問題を解く

補充すべき単語の推定 (id)

In [ ]:
# convert to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

# convert to tensor
import torch
token_tensor = torch.LongTensor([token_ids])

結果の表示

In [ ]:
# provide position ids explicitly
position_ids = list(range(0, token_tensor.size(1)))
position_id_tensor = torch.LongTensor([position_ids])

# get the top 10 predictions of the masked token
with torch.no_grad():
    outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

## 2. GPT-2 による文書生成

### (1) rinna/japanese-gpt2 の利用

### モデルのダウンロード

* 大きなモデル



In [33]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

* 小さなモデル

In [ ]:
from transformers import T5Tokenizer, GPT2LMHeadModel

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = GPT2LMHeadModel.from_pretrained("rinna/japanese-gpt2-small")

### 文書生成の例

In [ ]:
input = tokenizer.encode("私は防衛大学校に入校してからというもの、", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=3)
sentences = tokenizer.batch_decode(output)
for i in sentences:
    print(i)

## (2) GPT2-Japanese の利用

### モデルのダウンロードとインストール

In [ ]:
# gpt2-japaneseのインストール
!git clone https://github.com/tanreinama/gpt2-japanese
%cd gpt2-japanese
#!pip uninstall tensorflow -y
!pip install -r requirements.txt

In [ ]:
# smallモデルのダウンロード
!wget https://www.nama.ne.jp/models/gpt2ja-small.tar.bz2
!tar xvfj gpt2ja-small.tar.bz2

### ランダムな文章の作成

In [ ]:
# smallモデルの動作確認
!python gpt2-generate.py --model gpt2ja-small --num_generate 1

### 文章の続きを作成

In [ ]:
!python gpt2-generate.py --model gpt2ja-small --num_generate 3 --context="佐藤　浩は、"

In [ ]:
# データセットの作成
!git clone https://github.com/tanreinama/Japanese-BPEEncoder.git

In [ ]:
!python ./Japanese-BPEEncoder/encode_bpe.py --src_dir mydata --dst_file finetune

run_finetune.py を修正: 218行目を "for i in range(100):" にし、最後に save() を追加

In [ ]:
!python run_finetune.py --base_model gpt2ja-small --dataset finetune.npz --run_name gpr2ja-finetune_run1-small

In [ ]:
!python gpt2-generate.py --model checkpoint/gpr2ja-finetune_run1-small --num_generate 1